In [11]:
import OpenDartReader
import pandas as pd
import time
from pathlib import Path

api_key = '85c2367fdad752f376b3947ccdf9998092765f0e'  # OpenDart API KEY

In [2]:
dart = OpenDartReader(api_key)

In [3]:
stock_names = ['삼성전자', 'LG전자', '현대자동차']

In [4]:
stock_names

['삼성전자', 'LG전자', '현대자동차']

In [5]:
# 데이터 초기화. df2 는 정리된 데이터를 모으는 곳이며 얻고자 하는 데이터 이름을
# column 화 시킴.  빈 index 를 넣으면 추가가 안되므로 임으로 하나 넣자.
df2 = pd.DataFrame(columns=['유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

# '11013'=1분기보고서, '11012' =반기보고서, '11014'=3분기보고서, '11011'=사업보고서
reprt_code = ['11013', '11012', '11014', '11011']

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1900-01-01 to 1900-01-01
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   유동자산      0 non-null      object
 1   부채총계      0 non-null      object
 2   자본총계      0 non-null      object
 3   매출액       0 non-null      object
 4   매출총이익     0 non-null      object
 5   영업이익      0 non-null      object
 6   당기순이익     0 non-null      object
 7   영업활동현금흐름  0 non-null      object
 8   잉여현금흐름    0 non-null      object
dtypes: object(9)
memory usage: 80.0+ bytes


In [15]:
# 데이터를 얻기위한 반복문 시작
for stock_name in stock_names:
    # 정리 완료된 파일 저장하기 위한 경로 및 이름. result_종목이름.xlsx 형태로 저장된다.
    filePath = 'C:/workspace/capstone-finance/finance-data/data/'
    Path(filePath).mkdir(exist_ok=True)
    fileName = f'result_{str(stock_name)}.csv'
    for i in range(2015, 2022):  # OpenDart는 2015년부터 정보를 제공한다.

        # 더미 리스트 초기화. 1 ~ 4 분기 데이터를 합할 예정이므로 4 크기 만큼의 리스트 선언.
        current_assets = [0, 0, 0, 0]  # 유동자산
        liabilities = [0, 0, 0, 0]  # 부채총계
        equity = [0, 0, 0, 0]  # 자본총계
        revenue = [0, 0, 0, 0]  # 매출액
        grossProfit = [0, 0, 0, 0]  # 매출총이익 #grossMargin
        income = [0, 0, 0, 0]  # 영업이익 #EBIT
        net_income = [0, 0, 0, 0]  # 당기순이익
        cfo = [0, 0, 0, 0]  # 영업활동현금흐름
        cfi = [0, 0, 0, 0]  # 투자활동현금흐름
        fcf = [0, 0, 0, 0]  # 잉여현금흐름 : 편의상 영업활동 - 투자활동 현금흐름으로 계산

        for j, k in enumerate(reprt_code):
            df1 = pd.DataFrame()  # Raw Data
            if str(type(dart.finstate_all('004840', i, reprt_code=k, fs_div='CFS'))) == "<class 'NoneType'>":
                pass

            # 타입이 NoneType 이 아니면 읽어온다.
            else:
                df1 = df1.append(dart.finstate_all(stock_name, i, reprt_code=k, fs_div='CFS'))
                # 재무상태표 부분
                condition = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산')  # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계')  # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (
                        df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (
                                                                df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') |
                        (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (
                                df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') |
                        (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익') | (
                                df1.account_nm == '연결당기순이익') | (df1.account_nm == '연결분기(당기)순이익') | (
                                df1.account_nm == '연결반기(당기)순이익') |
                        (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))
                condition_9 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                current_assets[j] = int(df1.loc[condition].iloc[0]['thstrm_amount'])
                liabilities[j] = int(df1.loc[condition_2].iloc[0]['thstrm_amount'])
                equity[j] = int(df1.loc[condition_3].iloc[0]['thstrm_amount'])
                revenue[j] = int(df1.loc[condition_4].iloc[0]['thstrm_amount'])
                grossProfit[j] = int(df1.loc[condition_5].iloc[0]['thstrm_amount'])
                income[j] = int(df1.loc[condition_6].iloc[0]['thstrm_amount'])
                net_income[j] = int(df1.loc[condition_7].iloc[0]['thstrm_amount'])
                cfo[j] = int(df1.loc[condition_8].iloc[0]['thstrm_amount'])
                cfi[j] = int(df1.loc[condition_9].iloc[0]['thstrm_amount'])
                fcf[j] = (cfo[j] - cfi[j])

                if k == '11013':  # 1분기.
                    path_string = str(i) + '-03-31'
                elif k == '11012':  # 2분기
                    path_string = str(i) + '-06-30'
                elif k == '11014':  # 3분기
                    path_string = str(i) + '-09-30'
                else:  # 4분기. 1 ~ 3분기 데이터를 더한다음 사업보고서에서 빼야 함
                    path_string = str(i) + '-12-30'
                    revenue[j] = revenue[j] - (revenue[0] + revenue[1] + revenue[2])
                    grossProfit[j] = grossProfit[j] - (grossProfit[0] + grossProfit[1] + grossProfit[2])
                    income[j] = income[j] - (income[0] + income[1] + income[2])
                    net_income[j] = net_income[j] - (net_income[0] + net_income[1] + net_income[2])
                    fcf[j] = fcf[j] - (fcf[0] + fcf[1] + fcf[2])

                # 데이터프레임에 저장하는 부분
                df2.loc[path_string] = [current_assets[j], liabilities[j], equity[j],
                                        revenue[j], grossProfit[j], income[j], net_income[j], cfo[j], fcf[j]]
                df2.tail()  # 데이터프레임 끝에 저장한다.
            # 너무 잦은 요청이 있을 경우 OpenDart API 측에서 IP 를 차단하니 텀을 둔다.
            time.sleep(0.1)

    # 원본 dataframe 에도 영향을 끼치게끔 (inplace=True) 첫 행 drop.
    # df2.drop(['1900-01-01'], inplace=True) # 첫 행 drop

    # 파일 저장. fileName 에 명시된 경로에 각 종목코드별로 다른 이름으로 저장
    df2.to_csv(filePath + fileName)

    # 그 다음 종목의 데이터를 얻기 위해 df2 재초기화
    df2 = pd.DataFrame(columns=['유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익', '당기순이익', '영업활동현금흐름', '잉여현금흐름'],
                       index=['1900-01-01'])  # 정리된 Data. 빈 index 를 넣으면 추가가 안되므로 하나 넣자.